In [11]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import requests

from ipywidgets import widgets
from IPython.display import display, HTML, FileLink

In [136]:
username = ''
password = ''
data = pd.DataFrame()

def login(*args):
    """Get the username and password from the login fields and retrieve the data."""

    global username, password

    pw_status.value = username
    username = user_field.value
    password = pw_field.value

    pw_status.value = "Loading..."
        
    retrieve_data()

def retrieve_data():
    """Retrieve the latest data from the server"""

    global data
    
    url = "https://s20aalt05web01.sky.blackbaud.com/2532Altru/ODataQuery.ashx?databasename=d32a1a5b-211a-4f58-bab1-36132004843f&AdHocQueryID=379fdd07-ded0-4a38-b183-7c0e49118619"

    session = requests.Session()
    session.auth = (username, password)
    
    r = session.get(url)

    data = pd.DataFrame(r.json()['value'])

    data["Arrival"] = pd.to_datetime(data["Arrival"])
    data["Departure"] = pd.to_datetime(data["Departure"])
    
    data["Start time"] = data.apply(create_start_time, axis=1)
    data["End time"] = data.apply(create_end_time, axis=1)
    data["Ticket type"] = data["Tickettype"]
    
    data = data[["Name", "Arrival", "Departure", "Program", "Category", "Location", "Ticket type", "Quantity", "Capacity", "Start time", "End time"]]
    
    login_output.clear_output()


user_label = widgets.Label('Username:', layout=widgets.Layout(width='75px'))
user_field = widgets.Text(layout=widgets.Layout(width='175px'))
pw_field = widgets.Password(layout=widgets.Layout(width='175px'))
pw_submit_button = widgets.Button(description='Login',layout=widgets.Layout(width='75px'))
pw_submit_button.on_click(login)
pw_status = widgets.Label('', layout=widgets.Layout(width='150px'))
pw_label = widgets.Label('Password:', layout=widgets.Layout(width='75px'))

pw_login_box = widgets.VBox([
    widgets.HBox([user_label, user_field],
                layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap')),
    widgets.HBox([pw_label,pw_field], 
                 layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap')),
    widgets.HBox([ pw_submit_button, pw_status])
])

login_output = widgets.Output()
display(login_output)

login_output = widgets.Output()
display(HTML('<h1>ASC Field Trip Planner</h1>'))
display(login_output)

with login_output:
    display(pw_login_box)
    

Output()

Output()

In [206]:
browse_date_picker = widgets.DatePicker()
browse_select_date_button = widgets.Button(description="Select")
browse_select_date_button.on_click(generate_schedule_from_browser)
browse_date_box = widgets.HBox([browse_date_picker,browse_select_date_button])

browse_output = widgets.Output(layout={'border': '1px solid black'})

interface = widgets.Tab(layout=widgets.Layout(width="500px"))
interface.children = [
     widgets.VBox([browse_date_box, browse_output]),
    widgets.DatePicker()
]
interface.titles = ["Schedule browser", "Find"]

def show_interface(*args):
    display(interface)

In [207]:
show_interface()

In [130]:
def create_start_time(row):
    date = row["Arrival"]
    if row.Starttime is None:
        return None
    
    return pd.Timestamp(date.year, date.month, date.day, int(row.Starttime[0:2]), int(row.Starttime[2:]))

def create_end_time(row):
    date = row["Arrival"]
    if row.Endtime is None:
        return None
    
    return pd.Timestamp(date.year, date.month, date.day, int(row.Endtime[0:2]), int(row.Endtime[2:]))

In [175]:
def get_date(df, date):
    """Return the field trip entries for the given date."""
    
    if isinstance(date, str):
        split = date.split('-')
        date = datetime.datetime(int(split[0]),int(split[1]),int(split[2])).date()

    return df[df.Arrival.dt.date == pd.Timestamp(date).date()]

def get_location(df, location):
    """Return the field trip entries for the given location."""
    
    return df[df.Location == location]

def decimal_time(date):
    "Return the time as a decimal number of hours"
    
    return date.hour + date.minute/60

In [193]:
def format_name(name):
    """Format the name of a given program."""
    
    if name == 'SCH - L - Amusement Park Physics STEM Lab':
        return "Amusement\nPark Physics"
    if name == 'SCH - L - Squid Dissection STEM Lab':
        return "Squid\nDissection"
    if name == 'SCH - D - Matter Matters':
        return "Matter Matters"
    if name == 'PS - School Shows':
        return "School Show"
    if name == 'SCH - D - Cooking Up a Storm':
        return "Cooking Up\na Storm"
    if name == 'SCH - L - Cow Eye Dissection STEM Lab':
        return "Cow Eye\nDissection"
    if name == 'SCH - D - Get Energized!':
        return 'Get Energized!'
    if name == 'PS - To Worlds Beyond':
        return 'To Worlds\nBeyond'
    if name == 'SCH - L - Splitting Molecules STEM Lab':
        return 'Splitting\nMolecules'
    if name == 'SCH - D - Space Exploration':
        return 'Space\nExploration'
    if name == 'SCH - L - Fetal Pig STEM Lab':
        return 'Fetal Pig\nDissection'
    if name == 'PS - Nightwatch':
        return 'Nightwatch'
    if name == 'SCH - D - Chemistry is a Blast!':
        return 'Chemistry\nis a Blast'
    if name == "SCH - D - Shocking, It's Science!":
        return "Shocking,\nIt's Science!"
    if name == "SCH - D - Get Fired Up!":
        return 'Get Fired Up!'
    
    return name

def get_color(name_colors, name):
    """Return a color for each unique school name"""
    
    if name not in name_colors:
         name_colors[name] = sb.color_palette("pastel", n_colors=10)[len(name_colors)]

    return name_colors[name]

def check_legend(legend_names, name):
    """Check if the given name has been added to the legend."""
    
    if name in legend_names:
        return None
    
    legend_names[name] = True
    return name

def generate_schedule(date):
    """Generate a schedule image and return it."""
    
    locations = {
        "Jack Wood Hall": 1,
        "Eureka Theater": 2,
        "Learning Lab": 3,
        "Green Classroom": 4,
        "Yellow Classroom": 5,
        "Sudekum Planetarium": 6
    }
    
    day = get_date(data, date)

    if len(day) == 0:
        return
        
    name_colors = {}
    legend_names = {}
    
    plt.clf()
    combo = day.groupby(["Name", "Program", "Location",  "Start time", "End time", "Capacity"]).sum(numeric_only=True).reset_index()
    
    for i, row in combo.iterrows():
        if row.Location is None:
            continue
    
        start = decimal_time(row["Start time"])
        end = decimal_time(row["End time"])
        duration = end - start
        plt.bar(locations[row.Location], duration, bottom=start, label=check_legend(legend_names, row.Name), color=get_color(name_colors, row.Name))
        plt.text(locations[row.Location], (start + end)/2, format_name(row.Program) + "\n("+str(row.Quantity)+"/" + str(row.Capacity)+ ")", ha='center', va='center', wrap=True)
    
    # Add public shows
    plt.bar(2, .5, bottom=12.5, color=(0.5, 0.5, 0.5))
    plt.text(2, 12.75, "Live Science", ha='center', va='center', wrap=True, color='white')
    
    plt.bar(6, .5, bottom=11.5, color=(0.5, 0.5, 0.5))
    plt.text(6, 11.75, "Public Show", ha='center', va='center', wrap=True, color='white')
    
    plt.bar(6, .5, bottom=13.25, color=(0.5, 0.5, 0.5))
    plt.text(6, 13.5, "Public Show", ha='center', va='center', wrap=True, color='white')
    
    plt.bar(6, .5, bottom=14.25, color=(0.5, 0.5, 0.5))
    plt.text(6, 14.5, "Public Show", ha='center', va='center', wrap=True, color='white')
    
    
    plt.legend(bbox_to_anchor=(0.5, -0.2), loc='lower center', ncol=2)
    
    plt.title("Field Trip Schedule: " + str(np.min(day.Arrival.dt.date)))
    plt.gca().invert_yaxis();
    plt.yticks([9, 10, 11, 12, 13, 14, 15], ["9 AM", "10 AM", "11 AM", "12 PM", "1 PM", "2 PM", "3 PM"]);
    plt.xticks([1, 2, 3, 4, 5, 6], ["Jack Wood\nHall", "Eureka\nTheater", "Learning\nLab", "Green\nClassroom", "Yellow\nClassroom", "Sudekum\nPlanetarium"]);
    
    fig = plt.gcf()
    fig.set_size_inches(10, 8)
    plt.tight_layout()

    return plt.gcf()
    # plt.savefig('schedules/' + str(date) + '.pdf', dpi=300)


In [205]:
def generate_schedule_from_browser(*args):
    """Use the date from the date picker to create a schedule"""
    display(browse_date_picker.value)
    
    browse_output.clear_output()
    with browse_output:
        display(generate_schedule(browse_date_picker.value))